<a href="https://colab.research.google.com/github/insoucyant/MachineLearning/blob/main/PyTorchNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#https://www.youtube.com/watch?v=Jy4wM2X21u0
#Import
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim # Gradient Descent, Adam
import torch.nn.functional as F # Relu, Sigmoid
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # MNSIST Dataset 28*28 =784
import torchvision.transforms as transforms

# Sections
* Imports
* Create Fully Connected Netowrk
* Hyperparameters
* Load Data
* Initialize Network
* Loss and optimizer
* Train Network
* Check accuracy on training & test to see how good our model is

In [3]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):  # Input size = 28*28=784,
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50) # Input size nodes to hidden layer of 50 modes
    self.fc2 = nn.Linear(50, num_classes) # Hidden layer 50 nodes to putput classes

  def forward(self,x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [4]:
model = NN(784,10)
x = torch.randn(64,784) # Minibatch size
print(model(x).shape)


torch.Size([64, 10])


# Set Device

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters

In [6]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 1

# Load Data

In [9]:
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform = transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root = 'dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize Network

In [10]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

# Loss and optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network

In [14]:
for epoch in range(num_epochs): # One epoch means that the network has seen all the images in teh dataset
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # Get to correct shape
    data =data.reshape(data.shape[0], -1)

    # Forward
    scores = model(data)
    loss =criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent or adam step
    optimizer.step()



# Check accuracy on training & test to see how good the model is

In [29]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0],-1)

      scores = model(x)
      # 64 images * 10,
      _, predictions = scores.max(1)
      num_correct += (predictions ==y).sum()
      num_samples += predictions.size(0)

    acc = float(num_correct)/float(num_samples)

    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()
  # return acc

In [30]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 56042/60000 with accuracy 93.40
Checking accuracy on test data
Got 9353/10000 with accuracy 93.53
